# Mandelbrot Set

The Mandelbrot set is a mathematical object that is infamous for its aesthetically pleasing representations that one can find on the Internet.
For example, if one opens [its Wikipedia page](https://en.wikipedia.org/wiki/Mandelbrot_set) one is greeted with the following image:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Mandel_zoom_00_mandelbrot_set.jpg/1280px-Mandel_zoom_00_mandelbrot_set.jpg" alt="Mandelbrot set" width="200" />

The Mandelbrot set is a very common example of a fractal: a mathematical object that is self-similar.
What this means is that if you zoom in on the border of the Mandelbrot set, you will eventually find patterns that resemble the original, zoomed out object.

We will now guide you through the code you need to write in order to create your very first representation of the Mandelbrot set.

## The Complex Plane

First things first, we need to familiarise ourselves a little bit with the context in which our calculations will be taking place.
The Mandelbrot set is drawn on the *complex plane*, which is the space where complex numbers live.

Complex numbers are mathematical entities much like real numbers or integers, except sometimes we have a harder time understanding what a complex number is because it is harder to “see it”, as opposed to, for example, the integer 3: you can pick up three pencils and realise that you are looking at a representation of the concept of three.

In mathematics, the *imaginary unit*, which is usually represented as $i$, is a number that is defined to be the square root of $-1$.
In APL, we can compute the square root of a number `n` with `n*.5`, so let us see what the square root of $-1$ is:

In [1]:
¯1*.5

0J1

For a while now, Dyalog has added support for complex numbers, so we can work directly with them.
The imaginary unit is `0J1` in Dyalog, as we have seen above.

Now that we know what the imaginary unit is, we can define the complex numbers: a complex number is any number of the form $a + bi$, where $a$ and $b$ are real numbers, the numbers we are used to dealing with.
In Dyalog, we write $a + bi$ as `aJb`:

In [2]:
2J3

2J3

If, instead, you have the values of `a` and `b` in two variables:

In [3]:
a ← 2

In [4]:
b ← 3

Then you cannot write `aJb` directly because that looks like a variable name:

In [5]:
aJb

VALUE ERROR: Undefined name: aJb
      aJb
      ∧


Instead, you either have to multiply `b` explicitly with `0J1` and add that to `a`:

In [6]:
a+0J1×b

2J3

Or you can use the circle primitive `○`, for which some of the left arguments are useful when dealing with complex numbers.
For example, the left argument `¯11` can be used to multiply a number by the imaginary unit:

In [7]:
¯11○b

0J3

So, our complex number could be written as

In [8]:
a+¯11○b

2J3

The circle primitive can also be used to break complex numbers apart.
The part that comes before the `J` is called the *real part* of the complex number, and the part that comes after the `J` is called the *imaginary part* of the complex number.
The left arguments `9` and `11` can be used to extract the real and imaginary parts of a complex number, respectively:

In [9]:
⎕← c ← 2J3

2J3

In [10]:
9○c

2

In [11]:
11○c

3

## Building the Set

Now that we understand the fundamental pieces that we will be juggling in order to build the Mandelbrot set, we can now look at the basic algorithm that is used to build it.
If the Mandelbrot set is a *set* in the complex plane, then it means the Mandelbrot set contains complex numbers in it.
In order to build the set, we just need to know what is the *rule* that we used to decide if a number is *in* the set or not.

### Iterative function

That rule is really simple and it will build on this little function:

In [12]:
F ← {⍺+⍵*2}

`F` is a dyadic function that squares the right argument and then adds the left argument:

In [13]:
1 F 3

10

But `F` can also receive complex numbers as arguments, because complex numbers can also be added, subtracted, multiplied and divided.
(Granted, the way those operations work may seem strange at first, but the operations looking strange to you doesn't mean they aren't valid! If you want, take a little detour to [Wikipedia's article on complex numbers](https://en.wikipedia.org/wiki/Complex_number) and, in particular, to the [section on the operations](https://en.wikipedia.org/wiki/Complex_number#Relations_and_operations).)

In [14]:
0J1 F 3

9J1

In [15]:
2J1 F 3

11J1

In [16]:
0J1 F 2J¯1

3J¯3

Before we tell you how we decide if a complex number is inside the Mandelbrot set, let us look at another operation that is useful.

### Magnitude

For real numbers, we are used to using `|` to determine the absolute value of a number:

In [17]:
|3 ¯3

3 3

The absolute value of a real number can also be seen as the *distance* between the number and 0.
If we look at it like that, it becomes easy to define the “*absolute value*” of a complex number, which is actually its magnitude.

Let us think about the complex number `3J6`, which in mathematical notation is $3 + 6i$.
This number is 3 units to the right of 0, and then the $+6i$ means we also go **up** 6 units:

```
       3J6
       / |
      /  |
     /   |
    /    |  
   /     |  
  /      |
 /       |
0--------3
```

Now that we have this incredible diagram we can compute the distance between `3J6` and `0`, we just have to apply the Pythagorean theorem: the distance between `3J6` and `0` is the hypotenuse of the right triangle drawn, which means we need to square the lengths of the sides, add them, and then compute the square root of that:

In [18]:
.5*⍨+/3 6*2

6.7082

This is exactly what the *magnitude* `|` does for us:

In [19]:
|3J6

6.7082

### To Be or Not To Be

We know what the magnitude of a complex number is, which is the final thing we needed in order to determine if a number is or is not inside the Mandelbrot set.

To determine if a number `c` is inside the Mandelbrot set, consider the sequence `|F⍣0⍨c`, `|F⍣1⍨c`, `|F⍣2⍨c`, `|F⍣3⍨c`, ...
 - if the sequence of magnitudes tends to infinity, `c` does *not* belong to the Mandelbrot set;
 - if the sequence of magnitudes “stays small”, then `c` belongs to the Mandelbrot set.

Notice that the expression `|F⍣n⍨c` is just a compact way of writing `|c F c F c F ... F c`:

In [20]:
|c

3.60555

In [21]:
|F⍣0⍨c

3.60555

In [22]:
|c F c

15.2971

In [23]:
|F⍣1⍨c

15.2971

In [24]:
|c F c F c

231.009

In [25]:
|F⍣2⍨c

231.009

You get the idea, right?
Notice that, for this particular example, the sequence looks like it is growing very fast, and thus we could guess that `2J3` is not in the Mandelbrot set.

The problem is that, for numbers closer to `0`, it is very, *very*, hard to predict if `c` is going to “explode” like `2J3` did or if it is going to stay close to `0`.
What is more, just because you know if a given value `c` is in or out of the Mandelbrot set, it doesn't mean you can predict if a number very close to `c` is in there or not:

In [26]:
c ← ¯.6853J.3

In [27]:
|F\60⍴c

0.748088 0.325256 0.707456 0.478024 0.659646 0.636673 0.559355 0.80092 0.288563 0.812609 0.213355 0.751353 0.388828 0.714904 0.545279 0.665535 0.730075 0.538768 0.907667 0.158809 0.757007 0.354521 0.718764 0.508976 0.673591 0.683273 0.56977 0.869619 0.253314 0.811695 0.315437 0.770612 0.463483 0.746917 0.642223 0.71635 0.898953 0.610071 1.1149 0.560899 0.848596 0.250019 0.809963 0.286535 0.762856 0.439305 0.733659 0.608492 0.692974 0.834998 0.565458 1.03375 0.329559 0.692939 0.283064 0.668493 0.392462 0.626708 0.510503 0.555073

The sequence above makes it look like `c` is inside the Mandelbrot set.
However, if we take a look at a number that is ever so slightly to the left of `c`, we get a sequence that suddenly starts growing *a lot*:

In [28]:
|F\60⍴c-0.0001

0.74818 0.325242 0.707587 0.478023 0.659852 0.636744 0.559714 0.801232 0.289043 0.813096 0.213257 0.751741 0.388625 0.715385 0.54513 0.666326 0.730213 0.540359 0.908956 0.161928 0.757278 0.355913 0.719214 0.510465 0.674208 0.685533 0.570299 0.872945 0.252212 0.81078 0.320312 0.770819 0.468427 0.747812 0.649126 0.717934 0.910656 0.612122 1.12067 0.596254 0.921492 0.287638 0.809114 0.395659 0.785947 0.551058 0.781754 0.779683 0.791574 1.17043 0.854363 1.14636 1.62932 2.64519 7.73714 59.2891 3515.37 1.23579E7 1.52717E14 2.33223E28

To be absolutely rigorous, we still do not know if `c` itself is inside the Mandelbrot set or not, we just looked at the initial values of the sequence.
Naturally, we can compute more terms of the sequence.
Let us check the 120th term in the sequence:

In [29]:
|F⍣120⍨c

0.617933

We computed 120 terms now and it still looks like `c` is in the Mandelbrot set.
Surely this must mean that `c` is in the Mandelbrot set, right?

Wrong!
If we compute some more terms, we see that the sequence for `c` also “explodes” if we give it enough time:

In [30]:
|F⍣170⍨c

2.21314E24

This poses a problem for us: for a given `c`, how can we know how many terms we should compute in order to check if `c` is in the Mandelbrot set or not?
The simple answer is: we can't.
In practice, we choose a predetermined amount of iterations to perform, we compute those iterations and then look at the final computed element.
We then take that as an approximation: if the final element has already exploded, then `c` is not in the Mandelbrot set.
If the final element has not exploded, we take that `c` to be in the Mandelbrot set.

## Sampling the Plane

We have defined all we are going to work with, so now we can start writing the code for our first pictorial representation of the Mandelbrot set.
The first step we need to take is sample the complex plane: this means that we need to build a grid of complex numbers and then we will figure out which of the points in the grid are and aren't in the Mandelbrot set.

### Building the Grid

We start this process by defining the length of the grid, in number of points to be considered:

In [31]:
n ← 5

We start with a small value for `n` because we are still figuring out the code we will be writing, and in case we make a mistake, it is preferable that the arrays we are dealing with are small – this makes it easier to debug our program.
When we are confident in our program we can increment the value of `n` to create a larger grid.

After defining the size of the grid, we need to create it.
We will be creating a grid in a square that has unit length, so we can start by splitting the side of the square in `n` parts:

In [32]:
⎕← ticks ← n÷⍨¯1+⍳n+1

0 0.2 0.4 0.6 0.8 1

Think of these numbers as ticks in the horizontal axis.
The plane we are working in is the *complex* plane, so the horizontal axis represents the real parts of the numbers in our grid and the vertical axis will represent the imaginary parts of the numbers in our grid.

Because we already have a set of ticks we can also use it for the ticks along the vertical axis.
We just have to combine the horizontal and vertical ticks, to create all complex numbers that have a real part in the horizontal ticks and an imaginary part in the vertical ticks.

The vertical ticks, corresponding to the imaginary parts, are simply the horizontal ticks multiplied by the imaginary unit:

In [33]:
0J1×ticks

0 0J0.2 0J0.4 0J0.6 0J0.8 0J1

Now we just need to combine all the horizontal ticks with the vertical ticks, but we can do so by using the *outer product* operator:

In [34]:
ticks∘.+0J1×ticks

0 0.0J0.2 0.0J0.4 0.0J0.6 0.0J0.8 0.0J1
0.2 0.2J0.2 0.2J0.4 0.2J0.6 0.2J0.8 0.2J1
0.4 0.4J0.2 0.4J0.4 0.4J0.6 0.4J0.8 0.4J1
0.6 0.6J0.2 0.6J0.4 0.6J0.6 0.6J0.8 0.6J1
0.8 0.8J0.2 0.8J0.4 0.8J0.6 0.8J0.8 0.8J1
1 1.0J0.2 1.0J0.4 1.0J0.6 1.0J0.8 1.0J1

The only thing that is slightly off is that the real parts are varying vertically and the imaginary parts are varying horizontally, but we can easily fix that with the *transpose* `⍉`:

In [35]:
⍉ticks∘.+0J1×ticks

0 0.2 0.4 0.6 0.8 1 
0J0.2 0.2J0.2 0.4J0.2 0.6J0.2 0.8J0.2 1J0.2
0J0.4 0.2J0.4 0.4J0.4 0.6J0.4 0.8J0.4 1J0.4
0J0.6 0.2J0.6 0.4J0.6 0.6J0.6 0.8J0.6 1J0.6
0J0.8 0.2J0.8 0.4J0.8 0.6J0.8 0.8J0.8 1J0.8
0J1 0.2J1 0.4J1 0.6J1 0.8J1 1J1

Finally, if we want the grid to really mimic the complex plane, we need the upper rows of the grid to be the ones with the larger imaginary values, and not the other way around.
As it stands right now, the first row of the grid has the numbers with imaginary part equal to 0 and the last row has the numbers with imaginary part equal to 1.
To fix this, all we have to do is use the *reverse first* function:

In [36]:
⎕← square ← ⊖⍉ticks∘.+0J1×ticks

0J1 0.2J1 0.4J1 0.6J1 0.8J1 1J1 
0J0.8 0.2J0.8 0.4J0.8 0.6J0.8 0.8J0.8 1J0.8
0J0.6 0.2J0.6 0.4J0.6 0.6J0.6 0.8J0.6 1J0.6
0J0.4 0.2J0.4 0.4J0.4 0.6J0.4 0.8J0.4 1J0.4
0J0.2 0.2J0.2 0.4J0.2 0.6J0.2 0.8J0.2 1J0.2
0 0.2 0.4 0.6 0.8 1

### Adjusting the grid

The next step lies in translating the grid and resizing it, so that it covers the area in which we know the Mandelbrot set lies.
This is relevant because we know that the Mandelbrot set lies within a square that spans from `¯2.1` and `0.9` in the horizontal axis, and `¯1.5` and `1.5` in the vertical axis.
In other words, we want the lower left corner of our grid to be `¯2.1J¯1.5` and the upper right corner to be `0.9J1.5`.
Another possibility is to say that we want the length of the side of the grid to be `3` and the centre to be the point `¯0.6J0`.

Our grid currently has side length 1, so we can fix the side length by multiplying by the final side length:

In [37]:
3×square

0J3 0.6J3 1.2J3 1.8J3 2.4J3 3J3 
0J2.4 0.6J2.4 1.2J2.4 1.8J2.4 2.4J2.4 3J2.4
0J1.8 0.6J1.8 1.2J1.8 1.8J1.8 2.4J1.8 3J1.8
0J1.2 0.6J1.2 1.2J1.2 1.8J1.2 2.4J1.2 3J1.2
0J0.6 0.6J0.6 1.2J0.6 1.8J0.6 2.4J0.6 3J0.6
0 0.6 1.2 1.8 2.4 3

Finally, we have to fix the centre to be `¯0.6J0`.
The current centre of the grid is `1.5J1.5`, so what we can do is subtract its current centre and then add the new one:

In [38]:
¯0.6J0+¯1.5J¯1.5+3×square

¯2.1J01.5 ¯1.5J01.5 ¯0.9J01.5 ¯0.3J01.5 0.3J01.5 0.9J01.5
¯2.1J00.9 ¯1.5J00.9 ¯0.9J00.9 ¯0.3J00.9 0.3J00.9 0.9J00.9
¯2.1J00.3 ¯1.5J00.3 ¯0.9J00.3 ¯0.3J00.3 0.3J00.3 0.9J00.3
¯2.1J¯0.3 ¯1.5J¯0.3 ¯0.9J¯0.3 ¯0.3J¯0.3 0.3J¯0.3 0.9J¯0.3
¯2.1J¯0.9 ¯1.5J¯0.9 ¯0.9J¯0.9 ¯0.3J¯0.9 0.3J¯0.9 0.9J¯0.9
¯2.1J¯1.5 ¯1.5J¯1.5 ¯0.9J¯1.5 ¯0.3J¯1.5 0.3J¯1.5 0.9J¯1.5

Another possibility is recognising that subtracting a centre and adding a new one is the same as adding their difference:

In [39]:
¯0.6J0-1.5J1.5

¯2.1J¯1.5

We can now use their difference to shift the centre of the grid in a single operation:

In [40]:
⎕← grid ← ¯2.1J¯1.5+3×square

¯2.1J01.5 ¯1.5J01.5 ¯0.9J01.5 ¯0.3J01.5 0.3J01.5 0.9J01.5
¯2.1J00.9 ¯1.5J00.9 ¯0.9J00.9 ¯0.3J00.9 0.3J00.9 0.9J00.9
¯2.1J00.3 ¯1.5J00.3 ¯0.9J00.3 ¯0.3J00.3 0.3J00.3 0.9J00.3
¯2.1J¯0.3 ¯1.5J¯0.3 ¯0.9J¯0.3 ¯0.3J¯0.3 0.3J¯0.3 0.9J¯0.3
¯2.1J¯0.9 ¯1.5J¯0.9 ¯0.9J¯0.9 ¯0.3J¯0.9 0.3J¯0.9 0.9J¯0.9
¯2.1J¯1.5 ¯1.5J¯1.5 ¯0.9J¯1.5 ¯0.3J¯1.5 0.3J¯1.5 0.9J¯1.5

## Compute the Mandelbrot Set on the Grid

The next thing we need to do now is apply our function `F` to the whole grid.
Luckily, APL's array processing capabilities allow us to do so very easily:

In [41]:
⎕← magnitudes ← |F⍣9⍨grid

1.51111E179 8.04630E143 6.17622E114 4.86047E94 4.48927E113 5.3936E156
4.52893E134 6.11090E85 1.01550E48 5.01598E5 4.73253E34 5.7999E108
9.06608E83 2.43033E20 1.45282E0 3.27962E¯1 3.65442E¯1 1.7871E79 
9.06608E83 2.43033E20 1.45282E0 3.27962E¯1 3.65442E¯1 1.7871E79 
4.52893E134 6.11090E85 1.01550E48 5.01598E5 4.73253E34 5.7999E108
1.51111E179 8.04630E143 6.17622E114 4.86047E94 4.48927E113 5.3936E156

In the matrix above we can see that we already have some really large numbers, and that is why we only apply `F` 9 times.

After computing applying the iterative method, we need to figure out which values are inside the Mandelbrot set and which aren't, so we can just use a comparison:

In [42]:
2>magnitudes

0 0 0 0 0 0
0 0 0 0 0 0
0 0 1 1 1 0
0 0 1 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0

The `1`s above show points inside the Mandelbrot set and the `0`s show points outside.
To make a pictorial representation of the set, we could use indexing:

In [43]:
' #'[1+2>magnitudes]

### 
 ###

The representation above doesn't look very interesting, but bear with me for a second more.

## Recap

Let us take a step back and look at the code we have written so far:

 1. we created ticks in order to prepare for the grid:

In [44]:
n÷⍨¯1+⍳n+1

0 0.2 0.4 0.6 0.8 1

 2. we then used those ticks to create a grid with complex numbers:

In [45]:
t∘.+0J1×t←n÷⍨¯1+⍳n+1

0 0.0J0.2 0.0J0.4 0.0J0.6 0.0J0.8 0.0J1
0.2 0.2J0.2 0.2J0.4 0.2J0.6 0.2J0.8 0.2J1
0.4 0.4J0.2 0.4J0.4 0.4J0.6 0.4J0.8 0.4J1
0.6 0.6J0.2 0.6J0.4 0.6J0.6 0.6J0.8 0.6J1
0.8 0.8J0.2 0.8J0.4 0.8J0.6 0.8J0.8 0.8J1
1 1.0J0.2 1.0J0.4 1.0J0.6 1.0J0.8 1.0J1

 3. the grid was then readjusted, both in position and in alignment:

In [46]:
¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1

¯2.1J01.5 ¯1.5J01.5 ¯0.9J01.5 ¯0.3J01.5 0.3J01.5 0.9J01.5
¯2.1J00.9 ¯1.5J00.9 ¯0.9J00.9 ¯0.3J00.9 0.3J00.9 0.9J00.9
¯2.1J00.3 ¯1.5J00.3 ¯0.9J00.3 ¯0.3J00.3 0.3J00.3 0.9J00.3
¯2.1J¯0.3 ¯1.5J¯0.3 ¯0.9J¯0.3 ¯0.3J¯0.3 0.3J¯0.3 0.9J¯0.3
¯2.1J¯0.9 ¯1.5J¯0.9 ¯0.9J¯0.9 ¯0.3J¯0.9 0.3J¯0.9 0.9J¯0.9
¯2.1J¯1.5 ¯1.5J¯1.5 ¯0.9J¯1.5 ¯0.3J¯1.5 0.3J¯1.5 0.9J¯1.5

 4. after that, we applied our iterative function and found out the magnitudes of the values we computed:

In [47]:
|{⍺+⍵*2}⍣9⍨¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1

1.51111E179 8.04630E143 6.17622E114 4.86047E94 4.48927E113 5.3936E156
4.52893E134 6.11090E85 1.01550E48 5.01598E5 4.73253E34 5.7999E108
9.06608E83 2.43033E20 1.45282E0 3.27962E¯1 3.65442E¯1 1.7871E79 
9.06608E83 2.43033E20 1.45282E0 3.27962E¯1 3.65442E¯1 1.7871E79 
4.52893E134 6.11090E85 1.01550E48 5.01598E5 4.73253E34 5.7999E108
1.51111E179 8.04630E143 6.17622E114 4.86047E94 4.48927E113 5.3936E156

 5. finally, we used a comparison to turn those magnitudes into a pictorial representation of the Mandelbrot set:

In [48]:
' #'[1+2>|{⍺+⍵*2}⍣9⍨¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1]

### 
 ###

If the image doesn't look interesting, it is (in part) because our `n` was really small!
Now that it looks like our code is working, why don't you make it larger?

In [49]:
n ← 50
⎕← M ← ' #'[1+2>|{⍺+⍵*2}⍣9⍨¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1]

# 
 # 
 ## 
 ### 
 ###### 
 ##### 
 ##### 
 ## ######### # 
 ################# 
 ################ 
 ################# 
 ################## 
 # # #################### 
 ###### ################### 
 ############################ 
 ############################ 
 ############################## 
 ############################# 
 ####################################### 
 ############################# 
 ############################## 
 ############################ 
 ############################ 
 ###### ################### 
 # # #################### 
 ################## 
 ################# 
 ################ 
 ################# 
 ## ######### # 
 ##### 
 ##### 
 ###### 
 ### 
 ## 
 # 
 #

Doesn't that look much nicer?
I am not saying it competes with the Wikipedia image from the beginning, but take a look at the amount of code we wrote in order to get this picture!
We barely broke a sweat!

One of the things that looks odd about the representation above is its aspect ratio, because characters are generally taller than they are wide.
Duplicating each character horizontally can improve that slightly:

In [50]:
2/M

## 
 ## 
 #### 
 ###### 
 ############ 
 ########## 
 ########## 
 #### ################## ## 
 ################################## 
 ################################ 
 ################################## 
 #################################### 
 ## ## ######################################## 
 ############ ###################################### 
 ######################################################## 
 ######################################################## 
 ############################################################ 
 ########################################################## 
 ############################################################################## 
 ########################################################## 
 ############################################################ 
 ######################################################## 
 ######################################################## 
 ############ ###################################### 
 ## ## ######################################## 
 #################################### 
 ################################## 
 ################################ 
 ################################## 
 #### ################## ## 
 ########## 
 ########## 
 ############ 
 ###### 
 #### 
 ## 
 ##

## Next Steps

Now that you computed your very first Mandelbrot set, what should you do?
There are plenty of things you could try and I will leave you with some suggestions.

### Fiddle With the Numbers

The code we wrote creates a square grid of side length 3.
What happens if you change the side length?
What if you make it a rectangular grid instead of a square one?
What if you pick a different centre?

### Create Better Pictorial Representations

In order to be able to create more interesting representations of the Mandelbrot set, it is common to follow more or less the same workflow:
 - define a threshold, `t ← 2` for example;
 - define a maximum number of iterations, `iter ← 9` for example;
 - apply the iterative function a maximum of `iter` times to the grid;
 - determine what was the iteration number for which the magnitude of the point first exceeded the threshold;
 - do something with the information above.

Now, instead of only caring about whether or not a point blew up during the calculation of the sequence, we also care about *how quickly* each point blew up.

Using the small grid from before, here is what this could look like in code:

In [51]:
t ← 2

In [52]:
iter ← 9

In [53]:
grid

¯2.1J01.5 ¯1.5J01.5 ¯0.9J01.5 ¯0.3J01.5 0.3J01.5 0.9J01.5
¯2.1J00.9 ¯1.5J00.9 ¯0.9J00.9 ¯0.3J00.9 0.3J00.9 0.9J00.9
¯2.1J00.3 ¯1.5J00.3 ¯0.9J00.3 ¯0.3J00.3 0.3J00.3 0.9J00.3
¯2.1J¯0.3 ¯1.5J¯0.3 ¯0.9J¯0.3 ¯0.3J¯0.3 0.3J¯0.3 0.9J¯0.3
¯2.1J¯0.9 ¯1.5J¯0.9 ¯0.9J¯0.9 ¯0.3J¯0.9 0.3J¯0.9 0.9J¯0.9
¯2.1J¯1.5 ¯1.5J¯1.5 ¯0.9J¯1.5 ¯0.3J¯1.5 0.3J¯1.5 0.9J¯1.5

In [54]:
⎕← mat ← ⊃iter-+/2<|F\9⍴⊂grid

0 0 1 1 1 1
0 2 2 5 3 1
0 4 9 9 9 2
0 4 9 9 9 2
0 2 2 5 3 1
0 0 1 1 1 1

The matrix above gives you the number of the iteration for which that specific point was such that its magnitude was above `t`.
This means that a very small integer value means that that point blew up very quickly, whereas a larger integer value means that that point took longer to blow up.

The points for which their value equal `iter` were the ones whose magnitude never surpassed the threshold:

In [55]:
iter=mat

0 0 0 0 0 0
0 0 0 0 0 0
0 0 1 1 1 0
0 0 1 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0

This matches our first uninteresting pictorial representation of the Mandelbrot set:

In [56]:
' #'[1+iter=mat]

### 
 ###

#### Character-Based Representations

If you get creative with Unicode characters, you can produce the following:

In [57]:
iters ← iter-⊃+/2<|F\9⍴⊂¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1
' ░▒▓█'[(4÷⍨¯1+⍳5)⍸iters÷iter]

░░░░░░░ 
 ░░░░░▒█▒░░░░ 
 ░░░░░░▒██▒▒░░░░ 
 ░░░░░░░░▒▓██▒░░░░░ 
 ░░░░░░░░▒▓███▓▒░░░░░ 
 ░░░░░░░░▒██████▒░░░░░ 
 ░░░░░░░▒▒▒▓█████▒▒░░░░░ 
 ░░░░░░░▒▒▒▒▓██████▓▒▒▒▒░░░ 
 ░░░░░░░▒█████████████▓▒▒█▒░ 
 ░░░░░░░▒▒█████████████████▓░░ 
 ░░░░░░░░▒▒▓█████████████████▒░░ 
 ░░░░░░░░▒▒▓█████████████████▓▒░░ 
 ░░░▒▓▒▒▒▒▒▒▒▒▓██████████████████▓▒░░ 
 ░░░░░░▒█▓▒▓█▓▒▓▓████████████████████▒░░ 
 ░░░░░░░░▒▓██████▓█████████████████████▒░░ 
 ░░░░░░░░▒▒▓████████████████████████████▒░░░ 
 ░░░░░░░░▒▓▓████████████████████████████▓░░░░ 
 ░░░░░▒▒▒▓██████████████████████████████▒░░░░ 
 ░░▒▒▒▒▒▒▒▓██████████████████████████████▒░░░░ 
 ███████████████████████████████████████▓▒░░░░ 
 ░░▒▒▒▒▒▒▒▓██████████████████████████████▒░░░░ 
 ░░░░░▒▒▒▓██████████████████████████████▒░░░░ 
 ░░░░░░░░▒▓▓████████████████████████████▓░░░░ 
 ░░░░░░░░▒▒▓████████████████████████████▒░░░ 
 ░░░░░░░░▒▓██████▓█████████████████████▒░░ 
 ░░░░░░▒█▓▒▓█▓▒▓▓████████████████████▒░░ 
 ░░░▒▓▒▒▒▒▒▒▒▒▓██████████████████▓▒░░ 
 ░░░░░░░░▒▒▓█████████████████▓▒░░ 
 ░░░░░░░░▒▒▓█████████████████▒░░ 
 ░░░░░░░▒▒█████████████████▓░░ 
 ░░░░░░░▒█████████████▓▒▒█▒░ 
 ░░░░░░░▒▒▒▒▓██████▓▒▒▒▒░░░ 
 ░░░░░░░▒▒▒▓█████▒▒░░░░░ 
 ░░░░░░░░▒██████▒░░░░░ 
 ░░░░░░░░▒▓███▓▒░░░░░ 
 ░░░░░░░░▒▓██▒░░░░░ 
 ░░░░░░▒██▒▒░░░░ 
 ░░░░░▒█▒░░░░ 
 ░░░░░░░

Can you see what we are doing there?
We first calculated how much it takes each point to blow up.
Here is a section of those results:

In [58]:
⎕← section ← iters[20+⍳5;10+⍳5]

4 4 4 6 8
4 4 5 6 8
4 5 7 7 9
6 7 9 9 9
6 8 9 9 9

Then we need to transform the integers in the range `0` `1` ... `iter` into numbers that can be used to index into the character vector `' ░▒▓█'`, so we normalise the iteration counts to be decimal values between `0` and `1`:

In [59]:
section÷iter

0.444444 0.444444 0.444444 0.666667 0.888889
0.444444 0.444444 0.555556 0.666667 0.888889
0.444444 0.555556 0.777778 0.777778 1 
0.666667 0.777778 1 1 1 
0.666667 0.888889 1 1 1

At the same time, we split the interval $[0, 1]$ in 4 equal segments:

In [60]:
(4÷⍨¯1+⍳5)

0 0.25 0.5 0.75 1

Now we can use the *interval index* function to map the decimal values from before into an appropriate index for `' ░▒▓█'`, with the caveat that only points that never blew up get mapped to █:

In [61]:
' ░▒▓█'[(4÷⍨¯1+⍳5)⍸section÷iter]

░░░▒▓
░░▒▒▓
░▒▓▓█
▒▓███
▒▓███

#### Colour-Based Representations

In order to produce beautiful images like the one in the beginning of the notebook, we need to be able to map each point to a colour instead of to a character.

There are many ways to transform these integer values into colours.
The simplest one would be to assign each point to a shade of grey:

In [62]:
⌊255×iter÷⍨iter-mat

255 255 226 226 226 226
255 198 198 113 170 226
255 141 0 0 0 198
255 141 0 0 0 198
255 198 198 113 170 226
255 255 226 226 226 226

The matrix above would work if you were using integers from `0` to `255` (like in the RGB colour scheme) but you could also use values from `0` to `1`:

In [63]:
iter÷⍨iter-mat

1 1 0.888889 0.888889 0.888889 0.888889
1 0.777778 0.777778 0.444444 0.666667 0.888889
1 0.555556 0 0 0 0.777778
1 0.555556 0 0 0 0.777778
1 0.777778 0.777778 0.444444 0.666667 0.888889
1 1 0.888889 0.888889 0.888889 0.888889

You would then want to look at, for example, the documentation for `Bitmap` (available under Dyalog's documentation, "Object Reference" &#8680; "(A-Z)" &#8680; "Bitmap Object") to see how to create actual images and save them to files.

### Julia Set

There is a family of fractals that are very similar, in nature, to the Mandelbrot set, and those are called [Julia sets](https://en.wikipedia.org/wiki/Julia_set).
In order to draw a Julia set, you do the exact same thing that we have done for the Mandelbrot set, but instead of using the function `F` like we have seen, you bind it to a left argument:

In [64]:
⎕← M ← ' #'[1+2>|0J¯0.8∘F⍣9⊢¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1]

# 
 # 
 ### 
 ## #
 ###### # #
 ##### #####
 #### #######
 ##### #######
 ########### 
 ############## 
 ############# 
 ### ############# 
 ############# ## 
 ######### # 
 ######### 
 ####### 
 ###### 
 ######## 
 ####### 
 ######## 
 ###### 
 ####### 
 ######### 
 # ######### 
 ## ############# 
 ############# ### 
 ############# 
 ############## 
 ## ## ########### 
 ########### ##### 
 # ########## #### 
 ###### ##### 
 ## # ###### 
 # ## 
 ### 
 # 
 #

You could explore different left arguments to bind to `F` or perhaps look at some of the values listed under the “Examples” section on Wikipedia.

Another thing you could do is, can you redraw this Julia set with the block characters `' ░▒▓█'`?
It is not necessarily _hard_, but you need to tweak the respective code that works for the Mandelbrot set.

### Refine the Estimation

In our examples we applied our function `F` only 9 times.
Given the grid that we used, if we apply it just one more time we get a `DOMAIN ERROR`:

In [65]:
⎕← ' #'[1+2>|{⍺+⍵*2}⍣10⍨¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1]

DOMAIN ERROR
      ⎕←' #'[1+2>|{⍺+⍵*2}⍣10⍨¯2.1J¯1.5+3×⊖⍉t∘.+0J1×t←n÷⍨¯1+⍳n+1]
                      ∧


The problem is that some of the numbers along the edge of the grid are growing really fast, and at some point the values become too large for APL to represent them:

In [66]:
⊃grid

¯2.1J1.5

In [67]:
F⍣9⍨⊃grid

¯2.19233E178J1.49512E179

In [68]:
F⍣10⍨⊃grid

DOMAIN ERROR
F[0] F←{⍺+⍵*2}
           ∧


When creating coloured representations of these fractals, you generally want the gradient of the image to be smooth.
For that to happen, you need to be able to compute more terms of the iterative sequence, so you need to come up with a way to circumvent these `DOMAIN ERROR`s.
A possibility is to stop applying the iterative process to points whose magnitude is already too large.
How can you do that?